In [ ]:
!wget https://raw.githubusercontent.com/GL3MON/inpLSTM/main/input.txt
!wget https://github.com/GL3MON/inpLSTM/raw/main/LSTM_256.h5

--2024-02-29 04:09:23--  https://raw.githubusercontent.com/GL3MON/inpLSTM/main/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1155392 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.10M  --.-KB/s    in 0.04s   

2024-02-29 04:09:24 (29.5 MB/s) - ‘input.txt’ saved [1155392/1155392]

--2024-02-29 04:09:24--  https://github.com/GL3MON/inpLSTM/raw/main/LSTM_256.h5
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/GL3MON/inpLSTM/main/LSTM_256.h5 [following]
--2024-02-29 04:09:25--  https://raw.githubusercontent.com/GL3MON/inpLSTM/main/LSTM_256.h

In [ ]:
import sys
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

GATHERING DATA:
We read the input.txt file and convert everything to lowercase and store it in varaible raw_text

In [ ]:
def gatheringData(filename):
  filename = filename
  raw_text = open(filename, 'r', encoding='utf-8').read()
  raw_text = raw_text.lower()
  return raw_text

PRE-PROCESSING DATA:
We create a two dictionary which has letters mapped to it's respective integer token and vice-versa.
We create training data which has letterwise tokenized senetences as input(X) and the next letter after the sentence as the label(Y).
Then we do a test and evaluation split.

In [ ]:
def preProcessingData(raw_text):
  chars = sorted(list(set(raw_text)))
  char_to_int = dict((c, i) for i, c in enumerate(chars))
  int_to_char = dict((i, c) for i, c in enumerate(chars))
  n_chars = len(raw_text)
  n_vocab = len(chars)
  print("Total Characters: ", n_chars)
  print("Total Vocab: ", n_vocab)
  seq_length = 100
  dataX = []
  dataY = []
  for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
  n_patterns = len(dataX)
  print("Total Patterns: ", n_patterns)
  X = np.reshape(dataX, (n_patterns, seq_length, 1))
  X = X / float(n_vocab)
  Y = to_categorical(dataY)
  split = int(0.9*len(X))
  train_X = X[:split]
  eval_X = X[split:]
  train_Y = Y[:split]
  eval_Y = Y[split:]
  return train_X, train_Y, eval_X, eval_Y, char_to_int, int_to_char, n_vocab, dataX, dataY

CREATING MODEL:
We define a model containing "units" LSTM cells stacked up and passed to a dropout layer. Then the out of it softmaxed and obtained as vector with size of vocab_size

In [ ]:
def createModel(units, dropout):
  model = Sequential()
  model.add(LSTM(units, input_shape=(train_X.shape[1], train_X.shape[2])))
  model.add(Dropout(dropout))
  model.add(Dense(train_Y.shape[1], activation='softmax'))
  return model

TRAINING MODEL:
We use categorical crossentropy as loss function and Adam as optimizer. Then we call the fit function and train the model on the tokenized dataset.

In [ ]:
def trainingModel(model, optim, epochs, batch_size=128):
  model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['acc'])
  filepath="weights.hdf5"
  checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
  callbacks_list = [checkpoint]
  model.fit(train_X, train_Y, epochs=1, batch_size=128, callbacks=callbacks_list)

EVALUATION OF MODEL:
We just evaluate the model using the eval split we just split in the above functions.

In [ ]:
def evalModel(model, batch_size=128):
  results = model.evaluate(eval_X, eval_Y, batch_size=128)
  print("The test loss and test acc: ",results)

GENERATE PREDICTIONS:
We predict the next word in the senquence by processing the output vector of the LSTM model.

In [ ]:
def predictSentences(model):
  start = np.random.randint(0, len(dataX)-1)
  pattern = dataX[start]
  for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print(result,end="")
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
  print("\nDone.")

In [ ]:
raw_text = gatheringData("input.txt")
train_X, train_Y, eval_X, eval_Y, char_to_int, int_to_char, n_vocab, dataX, dataY = preProcessingData(raw_text)

Total Characters:  1115393
Total Vocab:  39
Total Patterns:  1115293


In [ ]:
# To Do train the model #2

In [ ]:
# To Do evaluate the model #3

In [ ]:
#To Do predict using the model #4

In [ ]:
#To Do save model #5